In [13]:
#checking the page
import fitz  # PyMuPDF
import re

# Open the PDF file
pdf_document = fitz.open("data.pdf")

# Extract text from the first page
page = pdf_document[158]
text = page.get_text("text")

# Regular expression to extract relevant data
# pattern = re.compile(r'(\d+)\s+([\d.]+\.\w+)\s+([A-Z0-9 \-\/]+?)\s+Spare Part\s+(\d+)\s+15%')
# pattern = re.compile(r'(\d+)\s+([\d.]+\.\w+)\s+([A-Z0-9 \-\/]+?)\s+Spare Part\s+(\d+)\s+15%', re.DOTALL)
# pattern = re.compile(r'(\d+)\s+([\d.]+\.\w+)\s+([A-Z0-9 \-\/]+?)\s+Spare Part\s+(\d+)\s+\d+%')
# pattern = re.compile(r'(\d+)\s+([\d.]+\.\w+)\s+(.+?)\s+Spare Part\s+(\d+)\s+\d+%')
# Regular expression to extract relevant data, considering both "Spare Part" and "Filter"
# pattern = re.compile(r'(\d+)\s+([\d.]+\.\w+)\s+(.+?)\s+(Spare Part|Filter|Battery)\s+(\d+)\s+\d+%')
# pattern = re.compile(r'(\d+)\s+([\w\d.]+\.\d+\.\d+)\s+(.+?)\s+(Spare Part|Filter|Battery)\s+(\d+)\s+\d+%')
# pattern = re.compile(r'(\d+)\s+([\w.]+\.\w+)\s+(.+?)\s+(Spare Part|Filter|Battery)\s+(\d+)\s+\d+%')
pattern = re.compile(r'(\d+)\s+([\w\d.]+)\s+(.+?)\s+(Spare Part|Filter|Battery|Oil)\s+(\d+)\s+\d+%')

# Find all matches in the text
matches = pattern.findall(text)

# Store extracted data in a dictionary
data = {}
for match in matches:
    sr_no, part_number, part_description, category, price = match
    data[sr_no] = {
        "Part_Number": part_number.strip(),
        "Part_Description": part_description.strip(),
        "Category": category.strip(),
        "Price_to_Customer": price.strip()
    }

# Print the extracted data for verification
print(f'length of pdf is {len(data)}')
# print(text)
for d in data:
    item = data[d]
    print(d)
    # print(item)
    # print(f'serial Number is {sr_no} number is {item["Part_Number"]} price is {item["Price_to_Customer"]} description is {item["Part_Description"]}')
    

length of pdf is 55
8691
8692
8693
8694
8695
8696
8697
8698
8699
8700
8701
8702
8703
8704
8705
8706
8707
8708
8709
8710
8711
8712
8713
8714
8715
8716
8717
8718
8719
8720
8721
8722
8723
8724
8725
8726
8727
8728
8729
8730
8731
8732
8733
8734
8735
8736
8737
8738
8739
8740
8741
8742
8743
8744
8745


In [14]:
# Escape first element

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import fitz  # PyMuPDF
import re

# Function to extract data from PDF using PyMuPDF
def extract_data_from_pdf(pdf_file):
    pdf_document = fitz.open(pdf_file)
    page = pdf_document[158]  # Assuming you want data from the first page
    text = page.get_text("text")

    # Regular expression to extract relevant data
    pattern = re.compile(r'(\d+)\s+([\w\d.]+)\s+(.+?)\s+(Spare Part|Filter|Battery|Oil)\s+(\d+)\s+\d+%')
    matches = pattern.findall(text)

    # Store extracted data in a dictionary indexed by serial number
    data = {}
    for match in matches:
        sr_no, part_number, part_description, category, price = match
        data[sr_no] = {
            "Part_Number": part_number.strip(),
            "Part_Description": part_description.strip(),
            "Category": category.strip(),
            "Price_to_Customer": price.strip()
        }

    return data

# Set up Selenium WebDriver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# URL of the login page and product page and Login credentials
username = "rajendra@palgroupnepal.com"
password = "rajendra@123"
login_url = "https://admin.palgroupnepal.com/login"
product_page_url = "https://admin.palgroupnepal.com/product"

# PDF file path
pdf_file = "data.pdf"

# Extract data from PDF
data = extract_data_from_pdf(pdf_file)

# Open the login page
driver.get(login_url)
time.sleep(2)

# Perform the login
driver.find_element(By.NAME, "email").send_keys(username)
time.sleep(1)
driver.find_element(By.NAME, "password").send_keys(password)
time.sleep(1)
driver.find_element(By.CSS_SELECTOR, "button[type='submit']").click()
time.sleep(3)

# Navigate to the product page
driver.get(product_page_url)
is_counter = False

# Iterate through all items in the data
successful_inserts = 0

for index, (sr_no, item) in enumerate(data.items()):
    # if index == 0 or index ==1:
    #     continue
    if is_counter:
            driver.get(product_page_url)

    try:
        print(f'Serial Number: {sr_no}, Part Number: {item["Part_Number"]}, Price: {item["Price_to_Customer"]}, Description: {item["Part_Description"]}')
        print('----------------------------------------------------------------------------->')
        
        button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".MuiButton-containedPrimary")))
        driver.execute_script("arguments[0].scrollIntoView(true);", button)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", button)
        
        # Fill in the form
        driver.find_element(By.CSS_SELECTOR, "input[id=':r18:']").send_keys(item["Part_Description"])
        driver.find_element(By.CSS_SELECTOR, "input[id=':r19:']").send_keys(item["Part_Number"])
        driver.find_element(By.CSS_SELECTOR, "input[id=':r1a:']").send_keys("N/A")
        driver.find_element(By.CSS_SELECTOR, "input[id=':r1b:']").send_keys(item["Price_to_Customer"])
        
        #Click the add button
        time.sleep(2)
        add_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.MuiBox-root.css-10igwnb button.MuiButton-containedPrimary")))
        driver.execute_script("arguments[0].scrollIntoView(true);", add_button)
        time.sleep(1)  # Give some time for the scroll to complete
        driver.execute_script("arguments[0].click();", add_button)
        time.sleep(1)  # Give some time for the scroll to complete

       
        
    
        is_counter = True
        successful_inserts += 1
        print(f"Data entered for serial number {sr_no} and part number {item['Part_Number']}")
    except KeyError:
        print(f"Key '{sr_no}' not found in data")
    except Exception as e:
        print(f"An error occurred: {e}")

print(f'Total data inserted: {successful_inserts}')
driver.quit()


Serial Number: 8691, Part Number: T4.206.18.0.00, Price: 274, Description: BRACKET FOR FUEL  PIPES
----------------------------------------------------------------------------->
Data entered for serial number 8691 and part number T4.206.18.0.00
Serial Number: 8692, Part Number: T4.206.23.0.PR, Price: 215, Description: JOINT
----------------------------------------------------------------------------->
Data entered for serial number 8692 and part number T4.206.23.0.PR
Serial Number: 8693, Part Number: T4.207.07.0.00, Price: 5151, Description: CAC PIPE
----------------------------------------------------------------------------->
Data entered for serial number 8693 and part number T4.207.07.0.00
Serial Number: 8694, Part Number: T4.207.08.0.00, Price: 714, Description: CAC HOSE
----------------------------------------------------------------------------->
Data entered for serial number 8694 and part number T4.207.08.0.00
Serial Number: 8695, Part Number: T4.207.09.0.00, Price: 6298, Desc

In [12]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
